In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import re
import json

In [3]:
df = pd.read_json('../housing-survey/data/craigslist.json')
df.head(10)

,address,price,bed,bath,area,company,neighborhood,laundry,pets,parking,utilities,property_type,year_built,description,images
0,3050 Birch Hollow Dr,1095,2.0,1.0,876,None,NaN,1,1,1,[],apartment,NaN,This bedroom apartments includes:\n\n→ Contemp...,[https://images.craigslist.org/00S0S_8ZkasrSZ6...
1,912 Rose Ave,2000,4.0,2.5,2000,None,NaN,1,1,1,[],house,NaN,"Stunning Ann Arbor home, perfect family house ...",[https://images.craigslist.org/00Y0Y_4tey2xf24...
10,3050 Birch Hollow Dr,1095,2.0,1.0,876,None,NaN,1,1,1,[],apartment,NaN,2 BD HOMESTEAD\nHomestead Commons Is In An Ide...,[https://images.craigslist.org/01010_2GBIdATh8...
100,2960 Birch Hollow Dr near School Stone Road,1179,2.0,1.0,812,None,NaN,1,1,0,[],apartment,NaN,This home is available NOW and we are offering...,[https://images.craigslist.org/00I0I_1f7yJKZFd...
1000,1207 Willard near Church,6300,7.0,0.0,0,None,NaN,1,0,1,[],apartment,NaN,This 7 bedroom house is perfect for any group....,[https://images.craigslist.org/00Y0Y_4N8yvCyzn...
1001,None,1076,1.0,1.0,846,None,NaN,1,1,0,[],apartment,NaN,Welcome home to Lynden Parke! # of Bedrooms: ...,[https://images.craigslist.org/00W0W_bKfxfWQrA...
1002,275 Harbor Way,1399,1.0,1.0,734,None,NaN,1,1,1,[],apartment,NaN,Amenities:\n⏩No Smoking\n⏩Sparkling Swimming P...,[https://images.craigslist.org/00t0t_4dp3rODyS...
1003,Stadium Blvd near Pauline Blvd,1199,2.0,1.0,670,None,NaN,1,1,1,[],apartment,NaN,**Images are from actual apartment**\n\nSign t...,[https://images.craigslist.org/00707_8dUFOFlDH...
1004,None,1429,2.0,2.0,1000,None,NaN,1,1,1,[],apartment,NaN,Traver Ridge is Now Taking Reservations for Su...,[https://images.craigslist.org/00k0k_fm2dNoBfq...
1005,1006 Pontiac Trail near Wall Street,1500,2.0,1.0,1400,None,NaN,1,0,1,[],apartment,NaN,"Walk to Kerrytown, Farmers Market, Zingerman's...",[https://images.craigslist.org/00Z0Z_2GioebIdN...


In [4]:
def fix_address(address):
    if address != None:
        address = address.lower()
        near_issue = address.find("near")
        apt_issue = address.find("apt")
        if near_issue != -1:
            address = address[0:near_issue-1]
        if apt_issue != -1:
            address = address[0:apt_issue-1]
        return address
    
# and then call apply on the series to apply the function to each element in the series
df['address'] = df['address'].apply(fix_address)
df.head(10)

,address,price,bed,bath,area,company,neighborhood,laundry,pets,parking,utilities,property_type,year_built,description,images
0,3050 birch hollow dr,1095,2.0,1.0,876,None,NaN,1,1,1,[],apartment,NaN,This bedroom apartments includes:\n\n→ Contemp...,[https://images.craigslist.org/00S0S_8ZkasrSZ6...
1,912 rose ave,2000,4.0,2.5,2000,None,NaN,1,1,1,[],house,NaN,"Stunning Ann Arbor home, perfect family house ...",[https://images.craigslist.org/00Y0Y_4tey2xf24...
10,3050 birch hollow dr,1095,2.0,1.0,876,None,NaN,1,1,1,[],apartment,NaN,2 BD HOMESTEAD\nHomestead Commons Is In An Ide...,[https://images.craigslist.org/01010_2GBIdATh8...
100,2960 birch hollow dr,1179,2.0,1.0,812,None,NaN,1,1,0,[],apartment,NaN,This home is available NOW and we are offering...,[https://images.craigslist.org/00I0I_1f7yJKZFd...
1000,1207 willard,6300,7.0,0.0,0,None,NaN,1,0,1,[],apartment,NaN,This 7 bedroom house is perfect for any group....,[https://images.craigslist.org/00Y0Y_4N8yvCyzn...
1001,None,1076,1.0,1.0,846,None,NaN,1,1,0,[],apartment,NaN,Welcome home to Lynden Parke! # of Bedrooms: ...,[https://images.craigslist.org/00W0W_bKfxfWQrA...
1002,275 harbor way,1399,1.0,1.0,734,None,NaN,1,1,1,[],apartment,NaN,Amenities:\n⏩No Smoking\n⏩Sparkling Swimming P...,[https://images.craigslist.org/00t0t_4dp3rODyS...
1003,stadium blvd,1199,2.0,1.0,670,None,NaN,1,1,1,[],apartment,NaN,**Images are from actual apartment**\n\nSign t...,[https://images.craigslist.org/00707_8dUFOFlDH...
1004,None,1429,2.0,2.0,1000,None,NaN,1,1,1,[],apartment,NaN,Traver Ridge is Now Taking Reservations for Su...,[https://images.craigslist.org/00k0k_fm2dNoBfq...
1005,1006 pontiac trail,1500,2.0,1.0,1400,None,NaN,1,0,1,[],apartment,NaN,"Walk to Kerrytown, Farmers Market, Zingerman's...",[https://images.craigslist.org/00Z0Z_2GioebIdN...


In [5]:
def fix_none(address):
    if address != None:
        return True
    else:
        return False

clean_df = df[df['address'].apply(fix_none)]
    
df['address'] = df['address'].apply(fix_address)
clean_df.head(10)

,address,price,bed,bath,area,company,neighborhood,laundry,pets,parking,utilities,property_type,year_built,description,images
0,3050 birch hollow dr,1095,2.0,1.0,876,None,NaN,1,1,1,[],apartment,NaN,This bedroom apartments includes:\n\n→ Contemp...,[https://images.craigslist.org/00S0S_8ZkasrSZ6...
1,912 rose ave,2000,4.0,2.5,2000,None,NaN,1,1,1,[],house,NaN,"Stunning Ann Arbor home, perfect family house ...",[https://images.craigslist.org/00Y0Y_4tey2xf24...
10,3050 birch hollow dr,1095,2.0,1.0,876,None,NaN,1,1,1,[],apartment,NaN,2 BD HOMESTEAD\nHomestead Commons Is In An Ide...,[https://images.craigslist.org/01010_2GBIdATh8...
100,2960 birch hollow dr,1179,2.0,1.0,812,None,NaN,1,1,0,[],apartment,NaN,This home is available NOW and we are offering...,[https://images.craigslist.org/00I0I_1f7yJKZFd...
1000,1207 willard,6300,7.0,0.0,0,None,NaN,1,0,1,[],apartment,NaN,This 7 bedroom house is perfect for any group....,[https://images.craigslist.org/00Y0Y_4N8yvCyzn...
1002,275 harbor way,1399,1.0,1.0,734,None,NaN,1,1,1,[],apartment,NaN,Amenities:\n⏩No Smoking\n⏩Sparkling Swimming P...,[https://images.craigslist.org/00t0t_4dp3rODyS...
1003,stadium blvd,1199,2.0,1.0,670,None,NaN,1,1,1,[],apartment,NaN,**Images are from actual apartment**\n\nSign t...,[https://images.craigslist.org/00707_8dUFOFlDH...
1005,1006 pontiac trail,1500,2.0,1.0,1400,None,NaN,1,0,1,[],apartment,NaN,"Walk to Kerrytown, Farmers Market, Zingerman's...",[https://images.craigslist.org/00Z0Z_2GioebIdN...
1006,2960 international drive,989,2.0,1.0,700,None,NaN,1,1,0,[],apartment,NaN,"We've the perfect 2 bedroom, 3rd floor home wi...",[https://images.craigslist.org/00101_6pXB7oMlQ...
1008,w liberty,1379,1.0,1.0,850,None,NaN,1,1,1,[],apartment,NaN,"This quiet, smoke-free community is located in...",[https://images.craigslist.org/01010_5Mo6AHDXg...


In [6]:
usable_df = clean_df[['address','price']]
usable_df.head(10)

,address,price
0,3050 birch hollow dr,1095
1,912 rose ave,2000
10,3050 birch hollow dr,1095
100,2960 birch hollow dr,1179
1000,1207 willard,6300
1002,275 harbor way,1399
1003,stadium blvd,1199
1005,1006 pontiac trail,1500
1006,2960 international drive,989
1008,w liberty,1379


In [7]:
usable_df.to_csv("cleaned_craigslist.csv",header=True)